In [6]:
# 恭喜，这是最新的merge文件
import os
import numpy as np
import pandas as pd
import utils.data_loader as data_loader
import utils.display as display
from tqdm.notebook import tqdm

In [7]:
display.configure_logging()
display.configure_pandas()
tqdm.pandas()

C:\Users\hkrep\anaconda3\envs\et\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [8]:
def set_status_index(pairs, df_trajectory=None):
    begin = df_trajectory['plate'].searchsorted(pairs['license'])
    end = df_trajectory['plate'].searchsorted(pairs['license'], side='right')
    temp = df_trajectory.iloc[begin:end]
    x = temp['timestamp'].searchsorted(pairs['begin_time'])
    y = temp['timestamp'].searchsorted(pairs['end_time'], side='right')
    return pd.Series({'start': begin + x, 'end': begin + y})


def set_status(row, status, new_seg, status_value=1):
    if row['start'] == row['end']:
        new_seg[row['end']] = 1
        return
    status[row['start']: row['end']] = [status_value for i in range(row['start'], row['end'])]
    new_seg[row['end']] = 1
    return

In [9]:
# Load CE default load ce_v5_30min.csv
df_ce = data_loader.load_ce()
df_ce.rename(columns={'licence': 'license'}, inplace=True)
# Load intersection of trajectory and od, named et list.
et_list = data_loader.load_trajectory_od_intersection()
# load trajectory default load all history trajectory.
original_trajectory = data_loader.load_trajectory()
# Load od default load common od, named et od.
df_od = data_loader.load_od()
df_od.rename(columns={'Licence': 'license'}, inplace=True)


24-Jul-20 11:48:33 - Loading data/ce/v5_30min.csv
24-Jul-20 11:48:34 - Loading data/history_trajectories.csv
24-Jul-20 11:49:09 - Loading data/transaction_common_201407.csv


In [15]:
#######################################################################################################################
# Function1: Merge Trajectory OD
# This cell add status column to trajectory data. This cell need 5 min.
charging_status_index = df_ce.progress_apply(set_status_index, df_trajectory=original_trajectory, axis=1)
od_status_index = df_od.progress_apply(set_status_index, df_trajectory=original_trajectory, axis=1)

# 使用start、end对生成status列
status = [0] * original_trajectory.shape[0]
new_seg = [0] * original_trajectory.shape[0]
charging_status_index.progress_apply(set_status, args=(status, new_seg), status_value=2, axis=1)
od_status_index.progress_apply(set_status, args=(status, new_seg), status_value=1, axis=1)

# 添加status列
original_trajectory['status'] = status
original_trajectory['new_seg'] = new_seg

path = 'data/trajectory_w_status.csv'
original_trajectory.loc[original_trajectory['plate'].isin(et_list)].to_csv(path, index=False)
print(original_trajectory['plate'].nunique())
#######################################################################################################################





664
